In [1]:
# tao df2
import re
import pandas as pd

path = 'C:/Users/ngocb/Downloads/test_rex.xlsx'
df_trans = pd.read_excel(path)
arr_set_data = set()
num_row_data=0
num_row_process = 0
for col in df_trans.columns:
	df_trans[col] = df_trans[col].astype(str)
	lst_string_column = df_trans[col].to_list()
	num_row_data += len(lst_string_column)
	string = '\n'.join(lst_string_column)
	#thiết lập pattern
	#pattern = re.compile(r'^(?:(?:\w|\s|\b|\n){0,2})(?:(?:[0-9]{2}\/){2}[0-9]{2}[\s][0-9:]+[\s][A-Z]+[^\S][0-9]+[-0-9]{2}[\s]+(?:F|P)[\s][0-9]+[\s][0-9A-Z]+[\s]*(?:VN07)[\s]+(?:[0-9A-Z\s\/#\\]){2,8}[\s]+[0-9-.]*[\s]*(?:EA|CS|NA|N\/A|#NA|#N\/A){0,1}(?:\s|\n|\b){0,50}(?:Tech=){1}[A-Z0-9\s]{10})', re.MULTILINE)
	pattern = re.compile(r'^(?=.*?(?:\d{2}\/){2}(?:\d{2}))(?=.*?(?:(?:\d{2}:){2}\d{2})(?=[\sA-Z]))(?=.*?(?:\d+-\d{1})(?=\s+(?:F|P)))(?=.*?(?:F|P)(?=\s\d{6,12}))(?=.*?(?:\d{6,12})(?=\s{1,5}[A-Z0-9]{6,14}))(?=.*?(?:[A-Z0-9]{6,14})(?=\s{1,20}VN07))(?=.*?(?<=VN07)(?:\s+[A-Z0-9#\/\\\s]{2,14})).{30,}(?:\s|\n|\b){0,50}(?:Tech=){1}[A-Z0-9]{2,15}', re.MULTILINE)
	pattern_check_right = re.compile(r'(?=.*?(?<=VN07)(;[A-Z0-9#\/\\\s]+))(?=.*?([0-9.,]+)(?=;(?:CS|EA)))(?=.*?(?<=(?:CS|EA))(;[A-Z0-9]{2,15}))')
	patern_check_left = re.compile(r'(?=.*?(?<=[A-Z])(;\d+-\d{1}))(?=.*?(?<=(?:\d{2}\/){2}\d{2})(;(?:\d{2}:){2}\d{2}))(?=.*?(?<=(?:F|P))(;\d{2,20}))(?=.*?(?:\d+;)(?=[A-Z0-9]*))')

	list_trans = pattern.findall(string)
	list_trans_remove_duplicate = set(list_trans)
	num_row_process += len(list_trans)
	for i, line in enumerate(list_trans_remove_duplicate):
		#bỏ ký tự \n, Tech=
		row = re.sub(r'(\n|Tech=)', '', line).strip()
		#tìm index VN07
		index_vn07 = re.search(r'VN07', row).span()[0]
		#cắt chuỗi thành 2 phần trái, phải. để xử lý space location
		left_row = row[:index_vn07]
		right_row = row[index_vn07:]
		# with left
		left = re.sub(r'\s{1,}', ';', left_row)
		#with right
		# right1 = re.sub(r'(?<=[A-Z0-9])(\s{1})(?=[0-9A-Z])', '', right_row)
		right2 = re.sub(r'\s{2,}', ';', right_row)
		right = re.sub(r'\s{1,}', '', right2)
		#thêm đơn vị cho RPM
		# regex tìm chữ P theo sau nó phải là dấu ; và trước nó cũng là dấu ;
		# regex tìm chữ F theo sau nó phải là dấu ; và trước nó cũng là dấu ;. Trường hợp CASXFR không có đơn vị
		pattern_p = re.compile(r'(?:(?<=;)(P)(?=;))')
		pattern_f = re.compile(r'(?:(?<=;)(F)(?=;))')
		match_p = pattern_p.search(left)
		match_f = pattern_f.search(left)
		arr_right = right.split(';')
		if (len(arr_right) == 4) and (match_p is not None):
			arr_right.insert(-1, 'EA')
			right = ';'.join(arr_right)
		elif (len(arr_right) == 4) and (match_f is not None):
			arr_right.insert(-1, 'CS')
			right = ';'.join(arr_right)
		#ket qua
		string_final = left + right
		#check kết quả thêm data vào arr kết quả
		match_right = pattern_check_right.match(right)
		match_left = patern_check_left.match(left)
		if match_left and match_right:
			arr_set_data.add(string_final)
	   
print('Tổng số dòng data ban đầu: ', num_row_data)
print('Tổng số dòng cần xử lý: ', num_row_process)
print('Tổng số dòng đã xử lý: ', len(arr_set_data))
print('Tổng số dòng data trùng nhau: ', num_row_process - len(arr_set_data))
arr_set_data

Tổng số dòng data ban đầu:  5240075
Tổng số dòng cần xử lý:  807865
Tổng số dòng đã xử lý:  801949
Tổng số dòng data trùng nhau:  5916


{'11/28/24;15:28:45;DEPSUL;0010009772009622676-6;F;80825367;43330395W0;VN07;C037;50;CS;BID513',
 '11/15/24;11:55:57;DEPSUL;0010009772009027873-0;F;80694412;43200395K9;VN07;CONT001;51;CS;BID245',
 '11/14/24;05:42:58;PICKUL;0010009772009002507-5;F;80813665;43180395N0;VN07;STAK;48;CS;BID376',
 '11/06/24;20:24:11;PICKUL;0010009772008952986-7;F;80779929;43110395K8;VN07;STDC;34;CS;BID372',
 '11/04/24;23:16:09;PICKUL;0030009772002766058-8;P;21029021;0395411435;VN07;DOOR18;2500;EA;BID413',
 '11/04/24;18:56:54;DEPSUL;0010009772008962765-5;F;80744312;43090395W2;VN07;CONT029;45;CS;BID276',
 '11/11/24;11:40:08;DEPSUL;0030009772002770555-5;P;21057253;0395412397;VN07;G602C;420;EA;BID844',
 '11/11/24;10:20:37;DEPSUL;0010009772009039296-2;F;80792031;43160395G4;VN07;ST11;56;CS;BID032',
 '11/04/24;23:38:46;PICKUL;0030009772002744172-9;P;21016922;0395407020;VN07;LB606C1;8500;EA;BID128',
 '11/09/24;04:54:15;PICKUL;0010009772008997533-6;F;80815422;43140395P0;VN07;WH2FK;40;CS;BID366',
 '11/20/24;13:35:03;DE

In [2]:
import pandas as pd
lst_total_2 = []
for line in arr_set_data:
    lst_line = line.split(';')
    lst_total_2.append(lst_line)

# transposed = list(zip(*lst_total))

columns = ['date', 'time', 'type', 'ulid', 'class', 'gcas', 'batch', 'plan', 'location', 'qty', 'unit', 'T#']
df_178687 = pd.DataFrame(lst_total_2, columns=columns)
# df_178687.to_excel('df.xlsx')



In [3]:
df_178687.sample(10)

,date,time,type,ulid,class,gcas,batch,plan,location,qty,unit,T#
634440,11/18/24,19:37:38,PICKUL,0010009772009008993-0,F,80778753,43230395V0,VN07,CONT028,45,CS,BID513
517267,11/01/24,11:45:43,DEPSUL,0010009772008967135-1,F,80796701,43060395Z1,VN07,C035,48,CS,BID379
494324,11/21/24,09:47:09,DEPSUL,0010009772009018197-9,F,80779873,43260395N2,VN07,STDM,36,CS,BID392
251658,11/28/24,06:19:43,PICKUL,0010009772009623929-2,F,80809581,43320395L0,VN07,FH01A,68,CS,BID032
344863,11/11/24,18:14:38,PICKUL,0010009772008996762-1,F,80796525,43160395O1,VN07,WH2FK,48,CS,BID844
337843,11/04/24,10:14:18,DEPSUL,0030009772002765511-9,P,21090355,0395411325,VN07,B518A,2500,EA,BID285
532551,11/13/24,00:27:40,DEPSUL,0030009772002771398-7,P,21221582,0395412587,VN07,FM36B,2500,EA,BID285
21578,11/19/24,05:48:40,DEPSUL,0010009772009013958-1,F,80773330,43240395W0,VN07,FH29A,36,CS,BID368
596306,11/13/24,09:57:59,PICKUL,0010009772008978156-2,F,80785631,43060395P3,VN07,FI07C,36,CS,BID324
589444,11/08/24,21:13:17,PICKUL,0010009772008988282-5,F,80819033,43130395E0,VN07,FH19A,44,CS,BID382


In [4]:
df_sort = df_178687.sort_values(['date', 'time'], ascending=True)
df_sort.tail(10)

,date,time,type,ulid,class,gcas,batch,plan,location,qty,unit,T#
178868,12/01/24,05:57:45,LINTAK,0010009772008990914-0,F,80809262,43350395N2,VN07,AK,40,CS,BID064
337872,12/01/24,05:57:48,LINTAK,0010009772008990913-3,F,80809262,43350395N2,VN07,AK,40,CS,BID064
260245,12/01/24,05:57:50,DEPSUL,0010009772008990913-3,F,80809262,43350395N2,VN07,STAK,40,CS,BID064
533231,12/01/24,05:57:50,DEPSUL,0010009772008990914-0,F,80809262,43350395N2,VN07,STAK,40,CS,BID064
516926,12/01/24,05:58:05,PICKUL,0010009772008990913-3,F,80809262,43350395N2,VN07,STAK,40,CS,BID384
706203,12/01/24,05:58:05,PICKUL,0010009772008990914-0,F,80809262,43350395N2,VN07,STAK,40,CS,BID384
419492,12/01/24,05:58:16,LINTAK,0010009772008991987-3,F,80813651,43360395L0,VN07,AE,50,CS,BID182
247201,12/01/24,05:58:26,DEPSUL,0010009772008991987-3,F,80813651,43360395L0,VN07,STAE,50,CS,BID182
248206,12/01/24,05:59:52,DEPSUL,0010009772008990914-0,F,80809262,43350395N2,VN07,FI25A,40,CS,BID384
321088,12/01/24,05:59:52,DEPSUL,0010009772008990913-3,F,80809262,43350395N2,VN07,FI25A,40,CS,BID384


In [9]:
check_columns = df_178687['class'].unique()
check_columns.sort()
check_columns

array(['F', 'P'], dtype=object)

In [10]:
df_178687[df_178687['ulid'] == '0010009772008759523-9']
#0010009772008683057-7

,date,time,type,ulid,class,gcas,batch,plan,location,qty,unit,T#


### So sánh 2 dataframe

In [53]:
# df_dff = pd.merge(left=df_180430,
#                   right=df_178687,
#                   on=['Time', 'Ulid'],
#                   how='left',
#                   indicator=True)


In [54]:
# df_khac = df_dff[df_dff['_merge'].isin(['left_only'])]
# # df_khac.to_excel('df_dff.xlsx')
# df_khac